In [1]:
# %% Imports & setup
import numpy as np
import pandas as pd
import customFunctions as fct
import foxes
import foxes.variables as FV
import foxes.input.farm_layout as layout
from foxes import algorithms
from foxes.output import FarmResultsEval

# %% 1) Read & clean the 2006 ERA5 time series
# (using your helper from customFunctions.py)
era5 = fct.read_ts_csv("reanalysis.csv")
# resample to hourly means, set datetime index
era5 = era5.resample("h", on="Time").mean()
era5.index = pd.to_datetime(era5.index)

# %% 2) Compute wind speed & direction (m/s and degrees)
era5["WS"] = np.sqrt(era5["u100"]**2 + era5["v100"]**2)
era5["WD"] = 180 + np.degrees(np.arctan2(era5["u100"], era5["v100"]))
# if TI is missing, insert a default 10 % turbulence intensity
if "TI" not in era5.columns:
    era5["TI"] = 0.10

# %% 3) Build FOXES TimeSeries states
states = foxes.input.states.Timeseries(
    data_source=era5,
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    var2col={FV.WS: "WS", FV.WD: "WD", FV.TI: "TI"},
    fixed_vars={FV.RHO: 1.225},
)

# %% 4) Create the wind farm layout for N-10.1
farm = foxes.WindFarm(name="N-10.1")
# read the geom CSV your colleagues used for Task 5
layout_data = pd.read_csv(
    "turbine-info/coordinates/area_of_interest/layout-N-10.1.geom.csv",
    index_col=None
)
# add turbines using the built-in NREL5MW model
layout.add_from_csv(
    farm,
    layout_data,
    turbine_models=["NREL5MW"],
    verbosity=0
)

# %% 5) Set up & run FOXES (internal wake only)
algo = algorithms.Downwind(
    farm,
    states,
    rotor_model="centre",
    wake_models=["Bastankhah2014_quadratic_ka02"],
    verbosity=0
)
results = algo.calc_farm()

# %% 6) Aggregate results & compute yield/efficiency
eval_ = FarmResultsEval(results)
eval_.add_capacity(algo)             # net capacity
eval_.add_capacity(algo, ambient=True)  # ambient capacity
eval_.add_efficiency()               # farm efficiency

# per-turbine annual yield [GWh], ambient and net
turbine_stats = eval_.reduce_states({
    FV.AMB_YLD: "weights",
    FV.YLD:     "weights",
    FV.EFF:     "weights"
})
farm_ambient = eval_.calc_mean_farm_power(ambient=True) / 1000  # MW
farm_net     = eval_.calc_mean_farm_power()          / 1000     # MW
farm_eff     = eval_.calc_farm_efficiency()
annual_yld   = turbine_stats[FV.YLD].sum()                     # GWh

# %% 7) Print summary & per-turbine table
print(f"Farm ambient power: {farm_ambient:.1f} MW")
print(f"Farm net power    : {farm_net:.1f} MW")
print(f"Farm efficiency   : {farm_eff:.2f}")
print(f"Annual yield      : {annual_yld:.2f} GWh")

print("\nPer-turbine results:")
print(turbine_stats[[FV.AMB_YLD, FV.YLD, FV.EFF]])


ModuleNotFoundError: No module named 'foxes'